# Dependencies

In [22]:

# !pip install opencv-python
# !pip install --upgrade pip
# !python3 -m pip install tensorflow
# !python3 -c "import tensorflow as tf; print(tf.reduce_sum(tf.random.normal([1000, 1000])))"
# !python -m pip install -U matplotlib
# !pip install pandas
# !pip install -U scikit-learn
# !pip install flask
# !pip install opencv-python==4.5.3.56


In [23]:
## Error popping up causing me to uninstall and reinstall numpy
# !pip3 uninstall -y numpy
# !pip3 install numpy

In [62]:
import matplotlib as mlp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


In [134]:
pd.show_versions()

/Users/haseebkhan/Desktop/dev/schoolClasses/CISC-4900/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



INSTALLED VERSIONS
------------------
commit           : 2e218d10984e9919f0296931d92ea851c6a6faf5
python           : 3.9.6.final.0
python-bits      : 64
OS               : Darwin
OS-release       : 21.6.0
Version          : Darwin Kernel Version 21.6.0: Mon Dec 19 20:44:01 PST 2022; root:xnu-8020.240.18~2/RELEASE_X86_64
machine          : x86_64
processor        : i386
byteorder        : little
LC_ALL           : None
LANG             : en_US.UTF-8
LOCALE           : en_US.UTF-8

pandas           : 1.5.3
numpy            : 1.24.2
pytz             : 2022.7.1
dateutil         : 2.8.2
setuptools       : 67.4.0
pip              : 23.0.1
Cython           : None
pytest           : None
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : None
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 3.1.2
IPython          : 8.10.0
pandas_datareader: None
bs4              : 

In [63]:
# import pathlib


test_mask = [] 
test_no_mask = []
# I would do this in a fancy os.loop way but I was using google colab and didn't really want to mess with the function too much
train_data_mask_dir="New_Masks_Dataset/Train/Mask"
train_data_no_mask_dir="New_Masks_Dataset/Train/Non_Mask"
test_data_mask_dir="New_Masks_Dataset/Test/Mask"
test_data_no_mask_dir="New_Masks_Dataset/Test/Non_Mask"


In [26]:
# print(train_imagePaths_mask)

In [153]:
# https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
# https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
import cv2
# from google.colab.patches import cv2_imshow
import os

IMAGE_HEIGHT  = 100
IMAGE_WIDTH = 100
NUM_CLASSES = 2


input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT)


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        # cv2_imshow(img)
        # Has to be like this because of Python Ambiguity
        if img is not None:
            grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(grayscale, input_shape)
            resized = np.reshape(resized, input_shape[0] * input_shape[1])
            images.append(resized)
    return images


In [169]:
train_mask = load_images_from_folder(train_data_mask_dir)
train_no_mask = load_images_from_folder(train_data_no_mask_dir)
test_mask = load_images_from_folder(test_data_mask_dir)
test_no_mask = (load_images_from_folder(test_data_no_mask_dir))

In [166]:
# type(train_mask[0])

numpy.ndarray

In [140]:
# print(train_mask)

In [141]:
# labels = {'No_Mask': 0, 'Mask': 1}


In [180]:
y_train_mask = [1] * len(train_mask)
y_train_no_mask = [0] * len(train_no_mask)
y_test_mask = [1] * len(test_mask)
y_test_no_mask = [0] * len(test_no_mask)


In [217]:
df_train = pd.DataFrame({'label': y_train_mask,'mask_data': train_mask})

In [218]:
df_train = df_train.append(pd.DataFrame({'label': y_train_no_mask, 'mask_data': train_no_mask}), ignore_index=True)

/var/folders/21/4ybwj4pj54x1sz2qhfgy1b9m0000gn/T/ipykernel_29339/1233198709.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(pd.DataFrame({'label': y_train_no_mask, 'mask_data': train_no_mask}), ignore_index=True)


In [219]:
df_train = df_train.sample(frac = 1)
df_train

,label,mask_data
217,1,"[220, 217, 216, 215, 215, 215, 214, 212, 211, ..."
216,1,"[202, 200, 199, 196, 200, 196, 199, 199, 197, ..."
37,1,"[166, 159, 180, 179, 182, 186, 188, 191, 193, ..."
328,0,"[185, 185, 185, 185, 185, 185, 185, 185, 185, ..."
322,0,"[88, 86, 84, 85, 85, 86, 86, 86, 87, 88, 89, 8..."
...,...,...
29,1,"[25, 69, 85, 110, 110, 121, 156, 174, 193, 201..."
329,0,"[195, 197, 200, 200, 199, 202, 202, 203, 203, ..."
587,0,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ..."
414,0,"[253, 255, 255, 255, 255, 255, 255, 255, 255, ..."


In [220]:
X_train = df_train.iloc[:, 1:] / 255.0
X_train = pd.DataFrame(X_train['mask_data'].tolist())
y_train = df_train.iloc[:, :1]

# .astype('float32')

In [221]:
df_test = pd.DataFrame({'label': y_test_mask,'mask_data': test_mask})
df_test = df_test.append(pd.DataFrame({'label': y_test_no_mask, 'mask_data': test_no_mask}), ignore_index=True)

df_test = df_test.sample(frac = 1)
df_test


/var/folders/21/4ybwj4pj54x1sz2qhfgy1b9m0000gn/T/ipykernel_29339/4284791088.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(pd.DataFrame({'label': y_test_no_mask, 'mask_data': test_no_mask}), ignore_index=True)


,label,mask_data
60,0,"[208, 198, 208, 209, 223, 222, 216, 212, 196, ..."
1,1,"[26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 2..."
81,0,"[53, 112, 137, 130, 63, 67, 71, 58, 26, 31, 52..."
8,1,"[57, 58, 61, 65, 67, 69, 69, 70, 68, 68, 68, 6..."
68,0,"[217, 219, 219, 220, 219, 220, 220, 220, 220, ..."
...,...,...
91,0,"[215, 220, 217, 222, 216, 216, 216, 215, 218, ..."
26,1,"[45, 41, 41, 34, 39, 50, 150, 159, 167, 156, 5..."
54,0,"[79, 80, 82, 86, 86, 88, 90, 88, 89, 91, 90, 9..."
66,0,"[98, 99, 99, 98, 99, 99, 99, 99, 99, 100, 100,..."


In [222]:
X_test = df_test.iloc[:, 1:] / 255.0
X_test = pd.DataFrame(X_test['mask_data'].tolist())
y_test = df_test.iloc[:, :1]


In [224]:
x_train = X_train.values.reshape(-1,100,100)
x_test = X_test.values.reshape(-1,100,100)

array([[array([0.89803922, 0.90196078, 0.90196078, ..., 0.65098039, 0.24313725,
               0.71764706])                                                    ],
       [array([0.6627451 , 0.65882353, 0.65882353, ..., 0.16862745, 0.15686275,
               0.13333333])                                                    ],
       [array([0.36078431, 0.36470588, 0.37647059, ..., 0.61176471, 0.57254902,
               0.56470588])                                                    ],
       [array([0.45490196, 0.58823529, 0.56078431, ..., 0.43529412, 0.45490196,
               0.50588235])                                                    ],
       [array([0.38823529, 0.37254902, 0.39607843, ..., 0.0627451 , 0.05098039,
               0.04705882])                                                    ],
       [array([0.29019608, 0.28627451, 0.27843137, ..., 0.80392157, 0.80784314,
               0.81176471])                                                    ],
       [array([0.40392157, 0

In [225]:
batch_size = 128
num_classes = 2
epochs = 15
# input image dimensions
input_shape = (100, 100, 1)


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



In [226]:
print(x_train.shape)
print(x_test.shape)

(599, 100, 100)
(99, 100, 100)


In [227]:

# The parameters I chose here were also from a project I did earlier, I'll link it here
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 49, 49, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 33856)             0         
                                                                 
 dropout_4 (Dropout)         (None, 33856)            

In [229]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test,y_test))

Epoch 1/15
5/5 [==============================] - 12s 1s/step - loss: 0.7295 - accuracy: 0.4808 - val_loss: 0.6740 - val_accuracy: 0.5657
Epoch 2/15
5/5 [==============================] - 8s 2s/step - loss: 0.6059 - accuracy: 0.6945 - val_loss: 0.5862 - val_accuracy: 0.7071
Epoch 3/15
5/5 [==============================] - 8s 2s/step - loss: 0.4859 - accuracy: 0.8164 - val_loss: 0.5037 - val_accuracy: 0.7879
Epoch 4/15
5/5 [==============================] - 9s 2s/step - loss: 0.3909 - accuracy: 0.8264 - val_loss: 0.4569 - val_accuracy: 0.8283
Epoch 5/15
5/5 [==============================] - 7s 1s/step - loss: 0.3753 - accuracy: 0.8264 - val_loss: 0.4218 - val_accuracy: 0.8384
Epoch 6/15
5/5 [==============================] - 7s 1s/step - loss: 0.3394 - accuracy: 0.8331 - val_loss: 0.4450 - val_accuracy: 0.7879
Epoch 7/15
5/5 [==============================] - 7s 1s/step - loss: 0.3209 - accuracy: 0.8481 - val_loss: 0.4518 - val_accuracy: 0.7879
Epoch 8/15
5/5 [========================

In [230]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.4278266131877899
Test accuracy: 0.7676767706871033


In [234]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"